In [1]:
import pandas as pd
pd.set_option("display.max_colwidth", 40)
import numpy as np
import re
#import spacy
import nltk
from nltk import FreqDist
import gensim
from gensim import corpora

# libraries for visualization
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

## Load  Data

In [2]:
df = pd.read_csv('AllReviewsCleanTokens.csv')

In [3]:
df.count()

Unnamed: 0    11324
date          11324
review        11324
rating        11324
title         11324
language      11324
trip_date     11324
hotel         11324
source        11324
tokens        11324
dtype: int64

In [4]:
#df['lemmantized_reviews'] = df['review'].apply(str.split)
df['tokens'] = df['tokens'].str.replace(r"[\[\]\']", "").str.split(", ")

In [5]:
data = df[['tokens','language']]

## Remove words

In [6]:
# As we can see, the clusters contain too many of these uninformative words so we remove them.
list_stop_words_en = ["one", "time", "could", "would", "get", "center","longleat","park","parks"]
list_stop_words_fr = ["tre", "plus", "si", "tous",'parcs','parc']
list_stop_words = list_stop_words_en + list_stop_words_fr

In [7]:
def remove_stop_word(x, list_stop_words=list_stop_words):
    clean_x = [l for l in x if l not in list_stop_words]
    return clean_x

In [8]:
data['tokens'] = data['tokens'].apply(remove_stop_word)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


## Building an LDA model

In [9]:
tokens_en = data[data.language == 'en'].tokens
tokens_fr = data[data.language == 'fr'].tokens

In [10]:
dictionary_en = corpora.Dictionary(tokens_en)
dictionary_fr = corpora.Dictionary(tokens_fr)

In [11]:
print('en:',len(dictionary_en),'\nfr:',len(dictionary_fr))

en: 18412 
fr: 5037


In [12]:
doc_term_matrix_en = [dictionary_en.doc2bow(rev) for rev in tokens_en]
doc_term_matrix_fr = [dictionary_fr.doc2bow(rev) for rev in tokens_fr]

In [13]:
import warnings
warnings.filterwarnings('ignore')

# Creating the object for LDA model using gensim library
LDA = gensim.models.ldamodel.LdaModel

# Build LDA model
num_topics = 5

lda_model_en = LDA(corpus=doc_term_matrix_en, id2word=dictionary_en, 
                num_topics=num_topics, 
                alpha=[0.0001] * num_topics, 
                eta=[0.0001] * len(dictionary_en),
                chunksize=2000,
                passes=4,
                random_state=100,
               )
lda_model_fr = LDA(corpus=doc_term_matrix_fr, id2word=dictionary_fr, 
                num_topics=num_topics, 
                alpha=[0.0001] * num_topics, 
                eta=[0.0001] * len(dictionary_fr),
                chunksize=2000,
                passes=4,
                random_state=100,
               )

In [14]:
lda_model_en.print_topics(num_words=8)

[(0,
  '0.017*"car" + 0.010*"go" + 0.009*"day" + 0.009*"take" + 0.009*"book" + 0.009*"dog" + 0.009*"train" + 0.008*"child"'),
 (1,
  '0.016*"go" + 0.012*"good" + 0.010*"food" + 0.010*"price" + 0.008*"activity" + 0.008*"book" + 0.007*"pool" + 0.007*"expensive"'),
 (2,
  '0.018*"great" + 0.015*"go" + 0.015*"year" + 0.013*"love" + 0.011*"family" + 0.011*"staff" + 0.011*"stay" + 0.010*"amp"'),
 (3,
  '0.011*"service" + 0.011*"clean" + 0.010*"villa" + 0.008*"stay" + 0.008*"go" + 0.008*"staff" + 0.007*"lodge" + 0.006*"quot"'),
 (4,
  '0.015*"good" + 0.012*"pool" + 0.010*"great" + 0.010*"well" + 0.010*"area" + 0.009*"lodge" + 0.009*"stay" + 0.008*"activity"')]

In [15]:
lda_model_fr.print_topics(num_words=8)

[(0,
  '0.020*"tre" + 0.015*"fair" + 0.012*"pass" + 0.011*"bien" + 0.011*"cent" + 0.010*"chos" + 0.010*"vrai" + 0.009*"sup"'),
 (1,
  '0.018*"tre" + 0.010*"bien" + 0.010*"beaucoup" + 0.009*"cher" + 0.008*"bon" + 0.008*"activit" + 0.008*"piscin" + 0.008*"pass"'),
 (2,
  '0.026*"tre" + 0.013*"enfant" + 0.012*"bien" + 0.011*"bon" + 0.009*"piscin" + 0.009*"pass" + 0.008*"activit" + 0.008*"vrai"'),
 (3,
  '0.034*"tre" + 0.014*"bien" + 0.013*"sejour" + 0.010*"piscin" + 0.009*"personnel" + 0.008*"cher" + 0.008*"propr" + 0.008*"hotel"'),
 (4,
  '0.016*"tre" + 0.011*"sall" + 0.011*"piscin" + 0.010*"bain" + 0.009*"bon" + 0.009*"chambr" + 0.009*"vill" + 0.008*"etre"')]

## Topic visualization

In [16]:
# Visualize the topics
pd.options.display.max_colwidth = 2000
viz = pyLDAvis.gensim.prepare(lda_model_en, doc_term_matrix_en, dictionary_en, mds='tsne')

In [17]:
pyLDAvis.enable_notebook()
viz
#pyLDAvis.show(viz)

PreparedData(topic_coordinates=            Freq  cluster  topics           x           y
topic                                                    
4      27.767532        1       1    2.272275  -83.575653
1      24.307474        1       2  123.048096  -81.356461
2      20.449324        1       3   63.739613  -14.225695
3      14.295362        1       4   36.811874 -165.944580
0      13.180302        1       5  129.358978 -170.711304, topic_info=     Category          Freq       Term         Total  loglift  logprob
term                                                                  
169   Default   2606.000000        car   2606.000000  30.0000  30.0000
1492  Default   1154.000000        dog   1154.000000  29.0000  29.0000
55    Default   3905.000000       love   3905.000000  28.0000  28.0000
158   Default   5979.000000       year   5979.000000  27.0000  27.0000
1761  Default   2440.000000        amp   2440.000000  26.0000  26.0000
1054  Default   2733.000000      train   2733.000000  25.0000  25.0000
45    Default   8211.000000      great   8211.000000  24.0000  24.0000
148   Default   3893.000000    service   3893.000000  23.0000  23.0000
187   Default   1297.000000      guest   1297.000000  22.0000  22.0000
1529  Default    816.000000      dirty    816.000000  21.0000  21.0000
219   Default   3552.000000       walk   3552.000000  20.0000  20.0000
171   Default   4853.000000      clean   4853.000000  19.0000  19.0000
196   Default   2254.000000      leave   2254.000000  18.0000  18.0000
4     Default   1641.000000      amaze   1641.000000  17.0000  17.0000
211   Default   5063.000000      staff   5063.000000  16.0000  16.0000
797   Default   2000.000000  fantastic   2000.000000  15.0000  15.0000
170   Default   4814.000000      child   4814.000000  14.0000  14.0000
960   Default   3585.000000      price   3585.000000  13.0000  13.0000
323   Default   1054.000000       tell   1054.000000  12.0000  12.0000
201   Default   3292.000000        old   3292.000000  11.0000  11.0000
1255  Default   1759.000000       land   1759.000000  10.0000  10.0000
311   Default   3119.000000       room   3119.000000   9.0000   9.0000
204   Default   2045.000000       quot   2045.000000   8.0000   8.0000
43    Default  10328.000000         go  10328.000000   7.0000   7.0000
829   Default   1144.000000   bathroom   1144.000000   6.0000   6.0000
116   Default    928.000000   customer    928.000000   5.0000   5.0000
626   Default   3608.000000     lovely   3608.000000   4.0000   4.0000
268   Default   4504.000000       food   4504.000000   3.0000   3.0000
36    Default   5091.000000     family   5091.000000   2.0000   2.0000
1196  Default   1788.000000    bedroom   1788.000000   1.0000   1.0000
...       ...           ...        ...           ...      ...      ...
196    Topic5    713.400024      leave   2254.010254   0.8760  -5.1521
1790   Topic5    144.965027    traffic    236.755402   1.5359  -6.7457
1201   Topic5    159.125504        cot    272.572998   1.4882  -6.6525
201    Topic5    856.409302        old   3292.710449   0.6797  -4.9694
154    Topic5   1110.776245       take   4989.538574   0.5242  -4.7094
1282   Topic5    303.971558    toddler    729.450439   1.1511  -6.0053
170    Topic5   1045.854980      child   4814.673828   0.4996  -4.7696
27     Topic5   1131.909790        day   5487.484375   0.4479  -4.6905
15     Topic5   1082.872925       book   6071.805176   0.3024  -4.7348
235    Topic5    672.025757     around   3183.714355   0.4709  -5.2119
164    Topic5    796.474121       area   4588.460938   0.2753  -5.0420
43     Topic5   1208.242920         go  10328.379883  -0.1193  -4.6253
51     Topic5    883.063110      lodge   5952.020020   0.1184  -4.9388
1303   Topic5    667.322083       bike   3686.118164   0.3174  -5.2189
57     Topic5    701.004272       make   4090.334473   0.2626  -5.1697
158    Topic5    844.982788       year   5979.042480   0.0697  -4.9829
459    Topic5    711.838196    holiday   4399.979980   0.2049  -5.1543
5

In [18]:
# Visualize the topics
pd.options.display.max_colwidth = 2000
viz = pyLDAvis.gensim.prepare(lda_model_fr, doc_term_matrix_fr, dictionary_fr, mds='tsne')

In [19]:
pyLDAvis.enable_notebook()
viz
#pyLDAvis.show(viz)

PreparedData(topic_coordinates=            Freq  cluster  topics           x           y
topic                                                    
2      39.882515        1       1 -141.958710 -269.095215
3      21.958525        1       2  221.635483  136.704285
0      16.827345        1       3  -94.500854  155.358292
1      12.831942        1       4 -356.930969  -21.869728
4       8.499675        1       5  162.545517 -185.533752, topic_info=     Category         Freq       Term        Total  loglift  logprob
term                                                                
34    Default   339.000000       sall   339.000000  30.0000  30.0000
6     Default   271.000000       bain   271.000000  29.0000  29.0000
77    Default  2503.000000        tre  2503.000000  28.0000  28.0000
20    Default   795.000000       fair   795.000000  27.0000  27.0000
69    Default   873.000000     piscin   873.000000  26.0000  26.0000
202   Default   512.000000       vill   512.000000  25.0000  25.0000
30    Default   507.000000     reserv   507.000000  24.0000  24.0000
13    Default   571.000000     chambr   571.000000  23.0000  23.0000
238   Default   577.000000       etre   577.000000  22.0000  22.0000
86    Default   743.000000   beaucoup   743.000000  21.0000  21.0000
473   Default   349.000000       lodg   349.000000  20.0000  20.0000
14    Default   752.000000       cher   752.000000  19.0000  19.0000
148   Default   848.000000       pass   848.000000  18.0000  18.0000
213   Default   820.000000     sejour   820.000000  17.0000  17.0000
292   Default   434.000000       fait   434.000000  16.0000  16.0000
72    Default   605.000000        sup   605.000000  15.0000  15.0000
10    Default   900.000000        bon   900.000000  14.0000  14.0000
36    Default   118.000000       saun   118.000000  13.0000  13.0000
178   Default   619.000000       chos   619.000000  12.0000  12.0000
153   Default   275.000000  recommand   275.000000  11.0000  11.0000
12    Default   613.000000       cent   613.000000  10.0000  10.0000
19    Default   377.000000        end   377.000000   9.0000   9.0000
46    Default   377.000000       week   377.000000   8.0000   8.0000
318   Default   379.000000       donc   379.000000   7.0000   7.0000
8     Default  1173.000000       bien  1173.000000   6.0000   6.0000
420   Default   412.000000        car   412.000000   5.0000   5.0000
121   Default   352.000000      visit   352.000000   4.0000   4.0000
7     Default   363.000000     besoin   363.000000   3.0000   3.0000
135   Default   450.000000    endroit   450.000000   2.0000   2.0000
360   Default    94.000000       quot    94.000000   1.0000   1.0000
...       ...          ...        ...          ...      ...      ...
69     Topic5    93.575363     piscin   873.306091   0.2316  -4.5101
238    Topic5    68.548645       etre   577.233643   0.3344  -4.8213
585    Topic5    10.216582      clair    28.996048   1.4220  -6.7248
19     Topic5    48.166912        end   377.522675   0.4062  -5.1741
384    Topic5    45.306900       deux   356.545410   0.4021  -5.2354
458    Topic5     7.769338      doubl    19.966166   1.5213  -6.9986
10     Topic5    78.904495        bon   900.017395   0.0310  -4.6806
30     Topic5    55.330482     reserv   507.111145   0.2497  -5.0355
46     Topic5    46.287273       week   377.535248   0.3663  -5.2140
318    Topic5    46.271984       donc   379.018555   0.3621  -5.2143
121    Topic5    43.504944      visit   352.826447   0.3720  -5.2759
420    Topic5    47.523762        car   412.417603   0.3043  -5.1876
134    Topic5    25.570183       egal   144.471130   0.7335  -5.8074
77     Topic5   135.318573        tre  2503.415527  -0.4526  -4.1412
14     Topic5    66.836700       cher   752.441650   0.0441  -4.8466
292    Topic5    48.517769       fait   434.274750   0.2734  -5.1669
78     Topic5    36.110878          2   279.265503   0.4196  -5.4622
86     Topic5    58.589218   beaucoup   743.068970  -0.0751  -4.9783
148    Topic5    62.043743      